## Kitchen training - test

In [20]:
import gdown
from sklearn.model_selection import train_test_split
import os
import shutil
import re

OD_PATH = '/models/research/object_detection/'
MAIN_PATH = '/walrus/kitchens_test/'
IMG_PATH = MAIN_PATH + 'images/'

### Resources

- [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
- [Tensorflow Object Detection API tutorial](https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10)

### Download data

Store all images and annotations in a single folder, zip the folder and store it in Google Drive under `safety/data/images/version_X` (replace the `X`).

In [4]:
file_id = "1N7bQ7xDUhkOnrB3TOKel6N7Us_ENeT5u"
gdown.download(url="https://drive.google.com/uc?id=" + file_id,
               output=IMG_PATH + 'images.zip',
               quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1N7bQ7xDUhkOnrB3TOKel6N7Us_ENeT5u
To: /walrus/kitchens_test/metrics.json
100%|██████████| 100k/100k [00:00<00:00, 1.19MB/s]


'/walrus/kitchens_test/metrics.json'

In [ ]:
!unzip ${IMG_PATH}images.zip -d $IMG_PATH

Randomly split data into train and test:
```
|-images
    |- train
    |- test
```
where the last level folders will contain both the images and annotations (.xml) files. (!!!!!!REMOVE + 'test' and _2 !!!!!)

In [29]:
os.mkdir(IMG_PATH + 'train')
os.mkdir(IMG_PATH + 'test')

files = [file[:-4] for file in os.listdir(IMG_PATH) if re.search('.*\.xml$', file)]
files_train, files_test = train_test_split(files, train_size=0.9, test_size=0.1, random_state=2014)

for file in files_train:
    shutil.move(IMG_PATH + file + '.xml', dst + file + '.xml')
    shutil.move(IMG_PATH + 'train/' + file + '.jpg', dst + file + '.jpg')
for file in files_test:
    shutil.move(IMG_PATH + file + '.xml', dst + file + '.xml')
    shutil.move(IMG_PATH + 'test/' + file + '.jpg', dst + file + '.jpg')

### Generate data

Generate training data to `./images/`

In [3]:
!echo {OD_PATH}xml_to_csv.py

/models/research/object_detection/xml_to_csv.py


In [4]:
!python {OD_PATH}xml_to_csv.py
print("")
!ls images -lh
print("")
!head {MAIN_PATH}images/train_labels.csv

Successfully converted xml to csv.
Successfully converted xml to csv.

total 405K
-rwxr-xr-x 1 root root  136 Nov  3 22:33 desktop.ini
dr-xr-xr-x 2 root root    0 Nov  4 14:37 test
-rwxr-xr-x 1 root root  39K Nov  4 14:44 test_labels.csv
dr-xr-xr-x 2 root root    0 Nov  4 14:38 train
-rwxr-xr-x 1 root root 364K Nov  4 14:44 train_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
ADE_train_00000594.jpg,653,512,chair,27,332,136,403
ADE_train_00000594.jpg,653,512,chair,247,349,340,437
ADE_train_00000594.jpg,653,512,chair,358,341,400,439
ADE_train_00000594.jpg,653,512,stove/oven,550,281,642,361
ADE_train_00000594.jpg,653,512,chair,61,315,126,386
ADE_train_00000594.jpg,653,512,stove/oven,442,310,555,328
ADE_train_00000595.jpg,700,473,stove/oven,254,291,442,346
ADE_train_00000595.jpg,700,473,stove/oven,504,232,594,346
ADE_train_00000595.jpg,700,473,dishwasher,346,259,401,298


Open `generate_tfrecord.py` and:
1. replace `import tensorflow` with `import tensorflow.compat.v1`.
2. Modify `class_test_to_int()` function with label-id mapping.

In [7]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record
print("")
!ls -lh | grep .*\.record$

/root/.local/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Successfully created the TFRecords: /walrus/kitchens_test/train.record
Successfully created the TFRecords: /walrus/kitchens_test/test.record

-rwxr-xr-x 1 root root  66M Nov  4 14:57 test.record
-rwxr-xr-x 1 root root 414M Nov  4 14:57 train.record


### Training

Create label map and save it in `training/labelmap.pbtxt` with the same mapping as in `generate_tfrecord.py`.

Get model checkpoint (other models in Tensorflow object detection's [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)).

In [8]:
!wget "http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz"
!tar xvzf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

--2019-11-04 14:57:19--  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.28.112, 2800:3f0:4005:400::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.28.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149119618 (142M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’

faster_rcnn_incepti 100%[===================>] 142.21M  1.81MB/s    in 84s     

2019-11-04 14:58:43 (1.69 MB/s) - ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [149119618/149119618]

faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28/mo

**Don't run for cards example**
1. Copy config file from `./samples/configs/` to `./training/`
2. Adapt parameters in config file:
    - `num_classes`
    - `fine_tune_checkpoint`
    - train_input_reader: `input_path`, `label_map_path`
    - `num_examples`: images/test/
    - eval_input_reader: `input_path`, `label_map_path`
    - `data_augmentation_options`, see augmentations and arguments in [preprocessor module](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto).

In [10]:
!cp {OD_PATH}samples/configs/faster_rcnn_inception_v2_pets.config training/faster_rcnn_inception_v2_kitchens.config

Move train.py from `legacy/` tom `.`

In [11]:
!cp {OD_PATH}legacy/train.py .

Train model (needs to be stopped manually)

In [7]:
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_kitchens.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






Instructions for updating:
Use object_detection/model_main.py.
W1106 21:08:01.119134 139943737583424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.

W1106 21:08:01.119478 139943737583424 module_wrapper.py:139] From train.py:90: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1106 21:08:01.122714 139943737583424 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:102: The name tf.gfile.G


W1106 21:08:01.973750 139943737583424 module_wrapper.py:139] From /models/research/object_detection/core/preprocessor.py:2937: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
W1106 21:08:01.998717 139943737583424 deprecation.py:323] From /models/research/object_detection/core/batcher.py:101: batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1106 21:08:02.003340 139943737583424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/

Instructions for updating:
Use keras.layers.flatten instead.
W1106 21:08:05.451374 139943737583424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1106 21:08:05.455605 139943737583424 regularizers.py:98] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1106 21:08:05.479192 139943737583424 regularizers.py:98] Scale of 0 disables regularizer.

W1106 21:08:05.762728 139943737583424 module_wrapper.py:139] From /models/research/object_detection/core/losses.py:177: The name tf.losses.huber_loss is deprecated. Please use tf.compat.v1.losses.huber_loss instead.


W1106 21:08:05.764245 139943737583424 module_wrapper.py:139] From /models/research/object_detection/core/losses.py:1

W1106 21:08:11.946959 139943737583424 variables_helper.py:157] Variable [FirstStageFeatureExtractor/InceptionV2/Mixed_3c/Branch_1/Conv2d_0b_3x3/BatchNorm/gamma/Momentum] is not available in checkpoint
W1106 21:08:11.947193 139943737583424 variables_helper.py:157] Variable [FirstStageFeatureExtractor/InceptionV2/Mixed_3c/Branch_1/Conv2d_0b_3x3/weights/Momentum] is not available in checkpoint
W1106 21:08:11.947257 139943737583424 variables_helper.py:157] Variable [FirstStageFeatureExtractor/InceptionV2/Mixed_3c/Branch_2/Conv2d_0a_1x1/BatchNorm/beta/Momentum] is not available in checkpoint
W1106 21:08:11.948374 139943737583424 variables_helper.py:157] Variable [FirstStageFeatureExtractor/InceptionV2/Mixed_3c/Branch_2/Conv2d_0a_1x1/BatchNorm/gamma/Momentum] is not available in checkpoint
W1106 21:08:11.948463 139943737583424 variables_helper.py:157] Variable [FirstStageFeatureExtractor/InceptionV2/Mixed_3c/Branch_2/Conv2d_0a_1x1/weights/Momentum] is not available in checkpoint
W1106 21:08:

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W1106 21:08:12.605009 139943737583424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/slim/python/slim/learning.py:742: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2019-11-06 21:08:13.323854: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-06 21:08:13.331479: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2773865000 Hz
2019-11-06 21:08:13.332263: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xe022c50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-06 21:08:13.332326: I tensorflow/compiler/xla/service/service.cc:176]   StreamE

### Export model

In [13]:
!ls -l training/

total 567122
-rwxr-xr-x 1 root root       269 Nov  4 16:18 checkpoint
-rwxr-xr-x 1 root root       136 Nov  4 14:50 desktop.ini
-rwxr-xr-x 1 root root  27840397 Nov  4 16:26 events.out.tfevents.1572880714.74a0ad80ad34
-rwxr-xr-x 1 root root      3574 Nov  4 15:16 faster_rcnn_inception_v2_kitchens.config
-rwxr-xr-x 1 root root  10466489 Nov  4 15:18 graph.pbtxt
-rwxr-xr-x 1 root root        79 Nov  4 15:09 labelmap.pbtxt
-rwxr-xr-x 1 root root 103009376 Nov  4 15:58 model.ckpt-1295.data-00000-of-00001
-rwxr-xr-x 1 root root     25561 Nov  4 15:58 model.ckpt-1295.index
-rwxr-xr-x 1 root root   5441806 Nov  4 15:58 model.ckpt-1295.meta
-rwxr-xr-x 1 root root 103009376 Nov  4 16:08 model.ckpt-1604.data-00000-of-00001
-rwxr-xr-x 1 root root     25561 Nov  4 16:08 model.ckpt-1604.index
-rwxr-xr-x 1 root root   5441806 Nov  4 16:08 model.ckpt-1604.meta
-rwxr-xr-x 1 root root 103009376 Nov  4 16:18 model.ckpt-1907.data-00000-of-00001
-rwxr-xr-x 1 root root     25561 Nov  4 16:18 model.ckpt-190

In [16]:
!python {OD_PATH}export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_kitchens.config --trained_checkpoint_prefix training/model.ckpt-1295 --output_directory inference_graph

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1104 16:28:48.156948 140301050382144 module_wrapper.py:139] From /models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1104 16:28:48.168290 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:389: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1104 16:28:48.169805 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W1104 16:28:48.25


W1104 16:28:53.766587 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:268: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W1104 16:28:53.766983 140301050382144 deprecation.py:323] From /models/research/object_detection/exporter.py:370: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W1104 16:28:53.770792 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:402: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README

234 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz


W1104 16:28:55.322115 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:419: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.


W1104 16:28:56.493039 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:332: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-04 16:28:56.493579: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-04 16:28:56.493635: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-11-04 16:28:56.493702: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (74a0ad80ad34): /proc/driver/nvidia/version does not exist
2019-11-04 16:28:56.494113: I tensorflow/core/platform


W1104 16:29:05.716187 140301050382144 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:188: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Writing pipeline config file to inference_graph/pipeline.config
I1104 16:29:05.716538 140301050382144 config_util.py:190] Writing pipeline config file to inference_graph/pipeline.config


In [17]:
!ls -lh inference_graph/

total 101M
-rwxr-xr-x 1 root root   77 Nov  4 16:28 checkpoint
-rwxr-xr-x 1 root root  136 Nov  4 14:50 desktop.ini
-rwxr-xr-x 1 root root  50M Nov  4 16:29 frozen_inference_graph.pb
-rwxr-xr-x 1 root root  50M Nov  4 16:28 model.ckpt.data-00000-of-00001
-rwxr-xr-x 1 root root  16K Nov  4 16:28 model.ckpt.index
-rwxr-xr-x 1 root root 1.8M Nov  4 16:28 model.ckpt.meta
-rwxr-xr-x 1 root root 3.1K Nov  4 16:29 pipeline.config
drwxrwxrwx 2 root root    0 Nov  4 16:29 saved_model


### Perform inference

In [18]:
!pwd

/walrus/kitchens_test


To `Object_detection_image.py`:
1. add before last line `cv2.imwrite('/path/to/save/boxed_image.JPG', image)` and comment last line out.
2. `NUM_CLASSES`
2. `IMAGE_NAME`

In [20]:
!cat Object_detection_image.py

If `NoneType` related error, reinstall opencv with `apt install opencv-python`.

In [21]:
!python Object_detection_image.py

Traceback (most recent call last):
  File "Object_detection_image.py", line 29, in <module>
    from utils import label_map_util
ModuleNotFoundError: No module named 'utils'
